# BQSKit Intermediate Representation (IR) Tutorial

This tutorial will introduce a user to the core objects in the BQSKit package, as well as, how to manipulate them.

## 1. Unitary Matrices

In BQSKit, most objects are part of the unitary class hierarchy. The base class, `Unitary`, represents a map from zero or more real numbers to a unitary matrix. All `Unitary` objects have a `get_unitary` method which takes in the unitaries parameters and returns the concrete `UnitaryMatrix` object.

In the next section, we will learn more about gates, but for now we will use two well known gates to learn about `Unitary` objects.

In [1]:
from bqskit.ir.gates import CNOTGate, RZGate

cnot = CNOTGate() # The constant Controlled-Not gate
rz = RZGate() # Z-Rotation gate that takes one angle as a parameter
print(cnot)
print(rz)

CNOTGate
RZGate


In this first exercise, let's ensure these gates are part of the unitary class hierarchy.

In [2]:
from bqskit.qis.unitary.unitary import Unitary

assert isinstance(cnot, Unitary)
assert isinstance(rz, Unitary)

All `Unitary` objects contain a `get_unitary` method that takes zero or more parameters and returns a `UnitaryMatrix`. This is because the BQSKit IR treats everything as a function from a vector of real numbers to a unitary matrix. 

**Exercise:** In the below example, we calculate the matrix for both gates. Try changing the parameters, what happens?

In [3]:
cnot_utry = cnot.get_unitary()
rz_utry = rz.get_unitary([0])

print("CNOT's Unitary")
print(cnot_utry)
print()

print("RZ's Unitary")
print(rz_utry)

CNOT's Unitary
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]]

RZ's Unitary
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]


We can ask the `Unitary` for the number of parameters it takes in its `get_unitary` method.

In [4]:
assert cnot.num_params == 0
assert rz.num_params == 1

# The cnot gate is constant since it takes zero parameters
assert cnot.is_constant()

# The rz gate is not constant and hence parameterized since it takes one or more parameters
assert rz.is_parameterized()

`Unitary` objects in BQSKit are always described in the context of a quantum system. As a result, we associate a `radixes` list with each object. This is a list of integers which describes the number of orthogonal states for each qudit in the system. For example, a n-qubit unitary would have a n-length list containing only `2`s. A qutrit unitary would have a list containing only `3`s, and hybrid systems are supported.

In [5]:
# NBVAL_IGNORE_OUTPUT
from bqskit.qis.unitary import UnitaryMatrix

# Generate a random unitary sampled from the Haar random distribituion
utry = UnitaryMatrix.random(4)
# print(utry)
print(f"Generated a {utry.num_qudits}-qubit unitary, radixes is: {utry.radixes}.")

# The random generation also takes an optional parameter `radixes`.
utry_qutrit = UnitaryMatrix.random(4, [3, 3, 3, 3])
print(f"Generated a {utry_qutrit.num_qudits}-qutrit unitary, radixes is: {utry_qutrit.radixes}.")

utry_mixed = UnitaryMatrix.random(3, [2, 3, 4])
print(f"Generated a {utry_mixed.num_qudits}-hybrid-qudit unitary, radixes is: {utry_mixed.radixes}.")

Generated a 4-qubit unitary, radixes is: (2, 2, 2, 2).
Generated a 4-qutrit unitary, radixes is: (3, 3, 3, 3).
Generated a 3-hybrid-qudit unitary, radixes is: (2, 3, 4).


In the last example, the `utry_mixed` unitary is associated with a 3-qudit system, where the first qudit is a qubit, the second is a qutrit, and last is a qudit with 4 bases.

**UnitaryMatrix**

`UnitaryMatrix` objects are concrete matrices that are also constant `Unitary` objects. They support most of the numpy api and for the most part can be used like normal numpy arrays. If you perform an operation that is closed for unitaries (conjugate, transpose, matmul, ...) the resulting object will remain a `UnitaryMatrix` object, otherwise it will downgrade to a `np.ndarray`.

In [6]:
# NBVAL_IGNORE_OUTPUT
from bqskit.qis.unitary import Unitary

assert isinstance(utry, Unitary)

import numpy as np
utry1 = UnitaryMatrix.random(2)
utry2 = UnitaryMatrix.random(2)
utry3 = utry1 @ utry2
utry4 = utry3.otimes(UnitaryMatrix.identity(1))
print(utry4)
assert isinstance(utry4, UnitaryMatrix)

[[ 0.30043982-0.06581517j -0.68307587-0.05520978j -0.02942948+0.35031251j
  -0.02922122-0.55796477j]
 [ 0.57362365+0.04410373j  0.50799221-0.14871057j -0.14057302-0.17182959j
  -0.4409986 -0.38088569j]
 [ 0.2022563 -0.2380398j   0.14448823-0.28739111j  0.8499012 +0.02402897j
   0.27015801-0.05535474j]
 [ 0.08462828+0.68533389j -0.30946541-0.22571881j  0.30530597-0.10491753j
  -0.42253614+0.3060743j ]]


## 2. Circuits, Gates, and Operations

Circuits are 2d arrays of operations. An operation encapsulates a gate, a set of qubits, and the gate's parameters if any. BQSKit supports many gates, you can find them [here](https://bqskit.readthedocs.io/en/latest/source/ir.html#bqskit-gates-bqskit-ir-gates). You will also find out how to implement them below if you would like to support your own gates.

In [7]:
from bqskit.ir import Circuit

# When creating a Circuit object, we just need to pass it how many qudits it will be for
circuit = Circuit(2) # Two-qubit circuit

# We can also pass in a `radixes` object similar to before
circuit = Circuit(4, [3, 3, 3, 3]) # Four-qutrit circuit

# You can also load a circuit from a qasm file:
# circuit = Circuit.from_file("some_program.qasm")

In [8]:
from bqskit.ir.gates import HGate, CXGate

# Let's construct a simple circuit that prepares a Bell state
circuit = Circuit(2)
circuit.append_gate(HGate(), 0) # Add a Hadamard gate on the 0-th qudit.
circuit.append_gate(CXGate(), (0, 1)) # Add a cnot on qudits 0 and 1 (Control 0).
print(circuit.get_unitary()) # Calculate and print the unitary for the circuit

[[ 0.70710678+0.j  0.        +0.j  0.70710678+0.j  0.        +0.j]
 [ 0.        +0.j  0.70710678+0.j  0.        +0.j  0.70710678+0.j]
 [ 0.        +0.j  0.70710678+0.j  0.        +0.j -0.70710678+0.j]
 [ 0.70710678+0.j  0.        +0.j -0.70710678+0.j  0.        +0.j]]


In [9]:
# NBVAL_IGNORE_OUTPUT

# We can also simulate a statevector:
print(circuit.get_statevector([1, 0, 0, 0]))
# This will return the output state when (1, 0, 0, 0) is given as the input state.

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [10]:
# We can iterate through all the operations in the circuit
for op in circuit:
    # This will print the entire operation
    # which includes the gate as well as
    # the qudits the gate is applied to
    # and the parameters if there are any
    print(op)

HGate@(0,)
CNOTGate@(0, 1)


BQSKit gates are simple objects that are designed to be simple to create, but also support a rich variety of features. Any gate that is created will need to specify a way to calculate its unitary through `get_unitary`. We also need to associate a quantum system with the gate, this specifies the `radixes` that the gate can handle. There are a number of gate base classes that handle implementing a lot of this for you. For example, the hadamard gate implementation is below:

In [11]:
import math

from bqskit.ir.gates.constantgate import ConstantGate
from bqskit.ir.gates.qubitgate import QubitGate
from bqskit.qis.unitary.unitarymatrix import UnitaryMatrix


class HGate(ConstantGate, QubitGate):
    _num_qudits = 1
    _qasm_name = 'h'
    _utry = UnitaryMatrix(
        [
            [math.sqrt(2) / 2, math.sqrt(2) / 2],
            [math.sqrt(2) / 2, -math.sqrt(2) / 2],
        ],
    )

Note, the gate subclasses both `ConstantGate` and `QubitGate`. A `ConstantGate` requires a `_utry` class or instance attribute and then will handle unitary calculations for you. The `QubitGate` requires a `_num_qudits` class or instance attribute and then will handle `radixes` calculations for you. There is also the `QutritGate`.

Parameterized gates, gates that take one or more parameters to calculate their unitary e.g. X-rotation gate, require a little more. Look at the below example to see how a X-rotation gate is implemented:

In [12]:
from typing import Sequence

import numpy as np

from bqskit.ir.gates.qubitgate import QubitGate
from bqskit.qis.unitary.differentiable import DifferentiableUnitary
from bqskit.qis.unitary.optimizable import LocallyOptimizableUnitary
from bqskit.qis.unitary.unitarymatrix import UnitaryMatrix
from bqskit.qis.unitary.unitary import RealVector
from bqskit.utils.cachedclass import CachedClass


class RXGate(
    QubitGate,
    DifferentiableUnitary,
    LocallyOptimizableUnitary,
    CachedClass,
):
    _num_qudits = 1
    _num_params = 1
    _qasm_name = 'rx'

    def get_unitary(self, params: RealVector = []) -> UnitaryMatrix:
        """Return the unitary for this gate, see :class:`Unitary` for more."""
        self.check_parameters(params)

        cos = np.cos(params[0] / 2)
        sin = -1j * np.sin(params[0] / 2)

        return UnitaryMatrix(
            [
                [cos, sin],
                [sin, cos],
            ],
        )

    def get_grad(self, params: RealVector = []) -> np.ndarray:
        """
        Return the gradient for this gate.

        See :class:`DifferentiableUnitary` for more info.
        """
        self.check_parameters(params)

        dcos = -np.sin(params[0] / 2) / 2
        dsin = -1j * np.cos(params[0] / 2) / 2

        return np.array(
            [
                [
                    [dcos, dsin],
                    [dsin, dcos],
                ],
            ], dtype=np.complex128,
        )

    def optimize(self, env_matrix: np.ndarray) -> list[float]:
        """
        Return the optimal parameters with respect to an environment matrix.

        See :class:`LocallyOptimizableUnitary` for more info.
        """
        self.check_env_matrix(env_matrix)
        a = np.real(env_matrix[0, 0] + env_matrix[1, 1])
        b = np.imag(env_matrix[0, 1] + env_matrix[1, 0])
        theta = 2 * np.arccos(a / np.sqrt(a ** 2 + b ** 2))
        theta *= -1 if b < 0 else 1
        return [theta]


This gate has a few methods. The `get_unitary` one is important and responsible for the calculation of its unitary. The `get_grad` is necessary since this is a `DifferentiableUnitary`. A differentiable unitary can be used in instantiation calls that use a gradient based optimization subroutine.

The `optimize` method is necessary since this is a `LocallyOptimizableUnitary`. This unitary can be used in instantiation calls that use a tensor-network based approach like QFactor.

By implementing a parameterized gate that is either differentiable or locally optimizable, your gate can then be used by bqskit algorithms in calls to instantiation during synthesis.

**Advanced Exercise:** Implement your own gate below. After learning about the synthesis options available with BQSKit in the next tutorial, attempt to synthesize a circuit with your gate. Look to see how other gates are implemented for inspiration.